In [37]:
# загружаем данные
from os import path, environ
import pandas as pd
import numpy as np

data_dir = "/home/olga/Documents/lastfm"
environ["DATA_DIR"] = "/home/olga/Documents/lastfm"

col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv ( ("lastfm_small.tsv"),
    sep="\t",
    header=None,
    names=col_names
)
data.head()

user  \
0  00000c289a1829a808ac09c00daf10bc3c4e223b   
1  00000c289a1829a808ac09c00daf10bc3c4e223b   
2  00000c289a1829a808ac09c00daf10bc3c4e223b   
3  00000c289a1829a808ac09c00daf10bc3c4e223b   
4  00000c289a1829a808ac09c00daf10bc3c4e223b   

                            artist-mbid           artist-name  total-plays  
0  3bd73256-3905-4f3a-97e2-8b341527f805       betty blowtorch         2137  
1  f2fb0ff0-5679-42ec-a55c-15109ce6e320             die Ärzte         1099  
2  b3ae82c2-e60b-4551-a76d-6620f1b456aa     melissa etheridge          897  
3  3d6bbeb7-f90e-4d10-b440-e153c0d10b53             elvenking          717  
4  bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8  juliette & the licks          706

In [2]:
# заполняем пустые значения
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
data["total-plays"] = data["total-plays"]
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

total-plays  user_id  artist_id
0         2137        1      15531
1         1099        1      63469
2          897        1      46858
3          717        1      15968
4          706        1      48969

In [3]:
data.describe()

total-plays         user_id       artist_id
count  1000000.000000  1000000.000000  1000000.000000
mean       216.606950    10232.925996    33678.492236
std        604.378024     5912.022447    19230.330182
min          1.000000        1.000000        1.000000
25%         34.000000     5118.000000    17298.000000
50%         94.000000    10237.000000    34544.000000
75%        225.000000    15347.000000    49488.000000
max     135392.000000    20465.000000    66799.000000

In [4]:
# разобьём наблюдения на тестовую и обучающую выборки
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [5]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 1
нет в тестовой выборке, но есть в обучающей: 32
всего пользователей: 20465


In [6]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

/home/olga/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [27]:
test_data[["user_id", "artist_id", "total-plays"]].to_csv("lastfm.test.0",
    sep="\t",
    header=False,
    index=False
)
test_file_name = "lastfm.test.0"

In [28]:
train_data[["user_id", "artist_id", "total-plays"]].to_csv("lastfm.train.0",
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"

In [7]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [8]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799694
Доля ненулевых элементов: 0.0005849441544937098
Среднее значение ненулевых элементов: 216.59454345703125
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


In [13]:
!pip install implicit

    100% |████████████████████████████████| 266kB 761kB/s ta 0:00:01
  Running setup.py bdist_wheel for implicit ... done
  Stored in directory: /home/olga/.cache/pip/wheels/9a/6e/e6/71106a349ec65d9024a2832d8ca94a2225c8c271b44bb742c8
Successfully built implicit


In [9]:
import time
from implicit.nearest_neighbours import CosineRecommender

model = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))
sparse_info(model.similarity)

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 2.9846725463867188 секунд
Размерности матрицы: (66800, 66800)
Ненулевых элементов в матрице: 1216120
Доля ненулевых элементов: 0.00027253576678977373
Среднее значение ненулевых элементов: 0.4437221865179849
Максимальное значение ненулевых элементов: 1.000000058256159
Минимальное значение ненулевых элементов: 0.0


In [80]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 28.189460039138794 секнуд


In [82]:
# используем mrec в качестве метрики качества
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs

[2017-12-08 02:17:19,622] INFO: processing /home/olga/Documents/lastfm/lastfm.test.0...
None
mrr            0.0502 +/- 0.0000
prec@5         0.0165 +/- 0.0000
prec@10        0.0151 +/- 0.0000
prec@15        0.0101 +/- 0.0000
prec@20        0.0076 +/- 0.0000


In [84]:
import implicit

In [111]:
#построим и обучим ALS
model2 = implicit.als.AlternatingLeastSquares(factors=50)

In [112]:
plays = coo_matrix((
    train_data["total-plays"].astype(np.double),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799896
Доля ненулевых элементов: 0.0005850919094089745
Среднее значение ненулевых элементов: 216.6265802054267
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


In [113]:
model2.fit(plays)


In [114]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(test_file_name + ".recs2.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model2.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 93.87595176696777 секнуд


In [115]:
# используем mrec в качестве метрики качества
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs2

[2017-12-08 03:09:42,283] INFO: processing /home/olga/Documents/lastfm/lastfm.test.0...
None
mrr            0.2942 +/- 0.0000
prec@5         0.1282 +/- 0.0000
prec@10        0.1070 +/- 0.0000
prec@15        0.0713 +/- 0.0000
prec@20        0.0535 +/- 0.0000


In [127]:
#для каждого артиста можем вывести похожих
related = model2.similar_items(70)
related

[(70, 0.99999999999999956),
 (63713, 0.96052580089083051),
 (48516, 0.94288776645303229),
 (10000, 0.93685512994891973),
 (7470, 0.93561800466686251),
 (7950, 0.93198317612730519),
 (55794, 0.92691325263065549),
 (1737, 0.92503123835581347),
 (25745, 0.92104056578436755),
 (65213, 0.92098645969254322)]

In [10]:

from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
TFIDFRecommender, bm25_weight)

In [21]:

tfidf = TFIDFRecommender()
bm25 = BM25Recommender(K1 = 100, B = 0.5)


In [23]:
bm25.fit(plays)

In [29]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(test_file_name + ".recs3.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in bm25.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 21.21807837486267 секнуд


In [36]:
# используем mrec в качестве метрики качества
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs3

[2017-12-09 01:53:52,760] INFO: processing /home/olga/Documents/lastfm/lastfm.test.0...
None
mrr            0.0682 +/- 0.0000
prec@5         0.0231 +/- 0.0000
prec@10        0.0182 +/- 0.0000
prec@15        0.0122 +/- 0.0000
prec@20        0.0091 +/- 0.0000


In [38]:
#также попробуем обучить tfidf
tfidf.fit(plays)

In [40]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(test_file_name + ".recs4.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in tfidf.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 24.20221447944641 секунд


In [44]:
# используем mrec в качестве метрики качества
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs4

[2017-12-09 02:11:47,588] INFO: processing /home/olga/Documents/lastfm/lastfm.test.0...
None
mrr            0.0543 +/- 0.0000
prec@5         0.0182 +/- 0.0000
prec@10        0.0133 +/- 0.0000
prec@15        0.0089 +/- 0.0000
prec@20        0.0067 +/- 0.0000
